# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Segmented Eratosthenes Sieve

In [1]:
import time

In [2]:
%load_ext Cython

# Algorithm

In [3]:
%%time
%%cython

from libc.stdlib cimport malloc, free

DEF LIMIT = 1024 * 31
DEF PRIME = 1024 * 4
DEF SIEVE = 1024 * 32

cdef inline int imin(int a, int b) nogil:
    return a if a < b else b

cdef inline int memset(char *p, int n) nogil:
    cdef:
        short *q = <short *>p
        int i, j = 0

    for i in range((n + 1) >> 1):
        j += q[i]
        q[i] = 0x0100

    return j >> 8

cdef int naive_sieve(char *sieve, int *primes, int *offsets, int n) nogil:
    cdef int i, j

    memset(sieve, n)

    for i in range(3, n, 2):
        if sieve[i]:
            j = i * i
            while j < n:
                sieve[j] = 0
                j += i << 1

            primes[0] = i
            offsets[0] = j
            primes += 1
            offsets += 1

    primes[0] = 0
    offsets[0] = 0

    return memset(sieve, n)

cdef int segmented_sieve(char *sieve, int *primes, int *offsets, int k, int n) nogil:
    cdef int i

    while primes[0]:
        i = offsets[0] - k
        while i < n:
            sieve[i] = 0
            i += primes[0] << 1
        offsets[0] = i + k

        primes += 1
        offsets += 1

    return memset(sieve, n)

cpdef int eratosthenes(int n) nogil:
    cdef:
        char *sieve
        int *primes
        int *offsets
        int k, total

    if n > LIMIT * LIMIT:
        return -1

    sieve = <char *>malloc(SIEVE)
    primes = <int *>malloc(PRIME * sizeof(int))
    offsets = <int *>malloc(PRIME * sizeof(int))

    total = naive_sieve(sieve, primes, offsets, imin(n, LIMIT))

    memset(sieve, SIEVE)
    k = LIMIT
    n -= LIMIT

    while n > 0:
        total += segmented_sieve(sieve, primes, offsets, k, imin(n, SIEVE))
        k += SIEVE
        n -= SIEVE

    free(sieve)
    free(primes)
    free(offsets)

    return total

CPU times: user 279 ms, sys: 36.2 ms, total: 315 ms
Wall time: 1.7 s


# Run

In [4]:
%%time
for i in range(1, 10):
    print('primes below 10**%d: %d' % (i, eratosthenes(10**i)))

primes below 10**1: 4
primes below 10**2: 25
primes below 10**3: 168
primes below 10**4: 1229
primes below 10**5: 9592
primes below 10**6: 78498
primes below 10**7: 664579
primes below 10**8: 5761455
primes below 10**9: 50847534
CPU times: user 1.27 s, sys: 6.46 ms, total: 1.28 s
Wall time: 1.29 s


## Timeit

In [5]:
%%time
%timeit eratosthenes(1024 * 31)
%timeit eratosthenes(10**6)
%timeit eratosthenes(10**7)
%timeit eratosthenes(10**8)
%timeit eratosthenes(10**9)

134 µs ± 68.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
919 µs ± 7.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.2 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
116 ms ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.31 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU times: user 44.2 s, sys: 136 ms, total: 44.3 s
Wall time: 46.6 s


# The End